In [1]:
import json
import pandas as pd
import requests
from bs4 import BeautifulSoup as bs

In [6]:

url = 'https://www.soccerbase.com/teams/home.sd'
r = requests.get(url)
soup = bs(r.content, 'html.parser')
teams = soup.find('div', {'class': 'headlineBlock'}, text='Team').next_sibling.find_all('li')

In [14]:
teams_dict = {}
for team in teams:
    link = 'https://www.soccerbase.com' + team.find('a')['href']
    team = team.text

    teams_dict[team] = link


In [15]:
teams_dict

{'Arsenal': 'https://www.soccerbase.com/teams/team.sd?team_id=142&comp_id=1',
 'Aston Villa': 'https://www.soccerbase.com/teams/team.sd?team_id=154&comp_id=1',
 'Bournemouth': 'https://www.soccerbase.com/teams/team.sd?team_id=359&comp_id=1',
 'Brentford': 'https://www.soccerbase.com/teams/team.sd?team_id=378&comp_id=1',
 'Brighton': 'https://www.soccerbase.com/teams/team.sd?team_id=381&comp_id=1',
 'Chelsea': 'https://www.soccerbase.com/teams/team.sd?team_id=536&comp_id=1',
 'Crystal Palace': 'https://www.soccerbase.com/teams/team.sd?team_id=646&comp_id=1',
 'Everton': 'https://www.soccerbase.com/teams/team.sd?team_id=942&comp_id=1',
 'Fulham': 'https://www.soccerbase.com/teams/team.sd?team_id=1055&comp_id=1',
 'Leeds': 'https://www.soccerbase.com/teams/team.sd?team_id=1524&comp_id=1',
 'Leicester': 'https://www.soccerbase.com/teams/team.sd?team_id=1527&comp_id=1',
 'Liverpool': 'https://www.soccerbase.com/teams/team.sd?team_id=1563&comp_id=1',
 'Man City': 'https://www.soccerbase.com/

In [19]:
a = "https://www.soccerbase.com/teams/team.sd?team_id=2848&comp_id=1"

In [23]:
r = requests.get('%s&teamTabs=results' % a)
soup = bs(r.content, 'html.parser')

In [24]:
soup.select('.score a em:first-child')

[<em>2</em>,
 <em>0</em>,
 <em>1</em>,
 <em>2</em>,
 <em>1</em>,
 <em>0</em>,
 <em>1</em>,
 <em>0</em>,
 <em>2</em>,
 <em>3</em>,
 <em>1</em>,
 <em>2</em>,
 <em>0</em>,
 <em>1</em>,
 <em>2</em>]

In [25]:
soup.select('.score a em + em')

[<em>1</em>,
 <em>0</em>,
 <em>0</em>,
 <em>1</em>,
 <em>1</em>,
 <em>0</em>,
 <em>0</em>,
 <em>3</em>,
 <em>0</em>,
 <em>0</em>,
 <em>0</em>,
 <em>1</em>,
 <em>4</em>,
 <em>1</em>,
 <em>3</em>]

In [26]:
team = []
comps = []
dates = []
h_teams = []
a_teams = []
h_scores = []
a_scores = []

consolidated = []
for equip, link in teams_dict.items():
    print('Acquiring %s data...' % equip)

    headers = ['Team', 'Competition', 'Home Team', 'Home Score', 'Away Team', 'Away Score', 'Date Keep']
    r = requests.get('%s&teamTabs=results' % link)
    soup = bs(r.content, 'html.parser')

    h_scores.extend([int(i.text) for i in soup.select('.score a em:first-child')])
    limit_scores = [int(i.text) for i in soup.select('.score a em + em')]
    a_scores.extend([int(i.text) for i in soup.select('.score a em + em')])

    limit = len(limit_scores)
    team.extend([equip for i in soup.select('.tournament', limit=limit)])
    comps.extend([i.text for i in soup.select('.tournament a', limit=limit)])
    dates.extend([i.text for i in soup.select('.dateTime .hide', limit=limit)])
    h_teams.extend([i.text for i in soup.select('.homeTeam a', limit=limit)])
    a_teams.extend([i.text for i in soup.select('.awayTeam a', limit=limit)])

Acquiring Arsenal data...
Acquiring Aston Villa data...
Acquiring Bournemouth data...
Acquiring Brentford data...
Acquiring Brighton data...
Acquiring Chelsea data...
Acquiring Crystal Palace data...
Acquiring Everton data...
Acquiring Fulham data...
Acquiring Leeds data...
Acquiring Leicester data...
Acquiring Liverpool data...
Acquiring Man City data...
Acquiring Man Utd data...
Acquiring Newcastle data...
Acquiring Nottm Forest data...
Acquiring Southampton data...
Acquiring Tottenham data...
Acquiring West Ham data...
Acquiring Wolves data...


In [27]:
df = pd.DataFrame(list(zip(team, comps, h_teams, h_scores, a_teams, a_scores, dates)), columns=headers)

In [28]:
df.head(50)

,Team,Competition,Home Team,Home Score,Away Team,Away Score,Date Keep
0,Arsenal,Premier League,Crystal Palace,0,Arsenal,2,2022-08-05 20:00
1,Arsenal,Premier League,Arsenal,4,Leicester,2,2022-08-13 15:00
2,Arsenal,Premier League,Bournemouth,0,Arsenal,3,2022-08-20 17:30
3,Arsenal,Premier League,Arsenal,2,Fulham,1,2022-08-27 17:30
4,Arsenal,Premier League,Arsenal,2,Aston Villa,1,2022-08-31 19:30
5,Arsenal,Premier League,Man Utd,3,Arsenal,1,2022-09-04 16:30
6,Arsenal,Europa League,FC Zurich,1,Arsenal,2,2022-09-08 17:45
7,Arsenal,Premier League,Brentford,0,Arsenal,3,2022-09-18 12:00
8,Arsenal,Premier League,Arsenal,3,Tottenham,1,2022-10-01 12:30
9,Arsenal,Europa League,Arsenal,3,Bodo Glimt,0,2022-10-06 20:00
